In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [8]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Import dataset from UCI.
#Dataset : SMS Spam data collection dataset from UCI.

link : https://archive.ics.uci.edu/ml/datasets/sms+spam+collection


In [2]:
from google.colab import files
upload=files.upload()

Saving SMSSpamCollection to SMSSpamCollection


Read data from File. Data has tab as separator and header of the files can be given as "Label" and "message". The label has 2 classes : 1. Spam 2. Ham representing Spam and normal msg respectively.


In [3]:
data=pd.read_csv("SMSSpamCollection",sep='\t',names=['label','message'])
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
data.shape

(5572, 2)

Let's define a function for cleaning the text and its preprocessing.
Preprocessing like converting all the text into lowercase, removing other characters like punctuation mark, digits etc from the message. Remove stop words from the message.

In [6]:
ps=PorterStemmer()
def data_preprocessing(message):
    message=message.lower()
    message=re.sub('[^a-z]',' ',message)
    message=message.split()
    message=[ps.stem(word) for word in message if not word in set(stopwords.words('english'))]
    message=' '.join(message)
    return message

Create corpus of our dataset.

In [13]:
corpus=[]
for i in range(len(data)):
  message=data_preprocessing(data['message'][i])
  corpus.append(message)
len(corpus)

5572

#Creating Bag Of Words using the corpus of the messages.
max_features is mainly used to select most frequent n features from BOW, as size of BOW can be very large but not all the words are important in that.


In [14]:
cv=CountVectorizer(max_features=5000) 
x=cv.fit_transform(corpus).toarray()	
x.shape

(5572, 5000)

In [17]:
x[2]

array([0, 0, 0, ..., 0, 0, 0])

creating label column after doing One-Hot Encoding and getting k-1 columns for k categories


In [15]:
y=pd.get_dummies(data['label'],drop_first=True)


#create train test dataset using Train_test_split.
use Stratify =y to have equal % all the classes of labels in train and test dataset .

In [19]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,stratify=y,random_state=25)


#Create Naive Bayes model.
Predict the output.

In [20]:
spam_detect_datamodel=MultinomialNB().fit(x_train,y_train)

y_pred=spam_detect_datamodel.predict(x_test)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#Classitication Report.
Lets check the accuracy of our model.

In [22]:
c=classification_report(y_test,y_pred)
print(c)

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1206
           1       0.90      0.95      0.92       187

    accuracy                           0.98      1393
   macro avg       0.95      0.97      0.96      1393
weighted avg       0.98      0.98      0.98      1393

